In [1]:
import numpy as np
from dateutil.relativedelta import relativedelta
import datetime

In [6]:
class BinomialTreeModel:
    def __init__(self, val_date, mat_date, coupon, payment_freq, risk_free_rate, vol, di, spot_price, 
                 conversion_ratio, recovery_value):
        
        self.duration = relativedelta(mat_date, val_date).years
        
        self.coupon = coupon
        self.payment_freq = payment_freq
        self.vol = vol
        self.di = di
        self.spot_price = spot_price
        self.conversion_ratio = conversion_ratio
        self.risk_free_rate = risk_free_rate
        self.recovery_value = recovery_value
        
    def _calculateParameters(self, vol, di, risk_free_rate, timestep):
        u = np.exp(np.sqrt((vol**2 - di) * timestep))
        d = 1/u

        a = np.exp((risk_free_rate) * timestep) 
        dft_prob = 1 - np.exp(-di * timestep)
        pu = (a - d*np.exp(-di*timestep))/(u - d)
        pd = (u*np.exp(-di*timestep) - a)/(u - d)

        return dft_prob, pu, pd, u, d

    def calculateValue(self, timesteps):
        self.dt = self.duration/timesteps
        self.timesteps = timesteps

        self.coupon_payment_step = -(-self.timesteps // (self.duration * self.payment_freq))

        self.dft_prob, self.pu, self.pd, self.u, self.d = self._calculateParameters(self.vol, self.di, self.risk_free_rate, self.dt)
        
        self.memoize = {i: dict() for i in range(101)}

        convertible_bond_value = self._calculateValue_helper(0, self.spot_price)
        return convertible_bond_value


    def _calculateValue_helper(self, t, stock_price):
        
        if stock_price in self.memoize[t]:
            return self.memoize[t][stock_price]
        
        convertible_value = stock_price * self.conversion_ratio
        if t == self.timesteps:
            return max(convertible_value, 100) + 100 * self.coupon / self.payment_freq
        else:
            val_up = self._calculateValue_helper(t+1, stock_price * self.u)
            val_down = self._calculateValue_helper(t+1, stock_price * self.d)

            val = (self.pu * val_up + self.pd * val_down + self.dft_prob * self.recovery_value * 100) * np.exp(-self.risk_free_rate * self.dt)
            
            # If this is a coupon payment timestep, add coupon payment to value
            if (t != 0) and (t % self.coupon_payment_step == 0):
                val = val + 100 * self.coupon / self.payment_freq
                
            to_return = max(convertible_value, val)
            self.memoize[t][stock_price] = to_return
            
            return to_return


In [7]:
val_date = datetime.date(2021, 11, 1)
mat_date = datetime.date(2026, 11, 1)

model = BinomialTreeModel(val_date, mat_date, 0.03, 2, 0.0125, 0.3, 0.01, 20, 3.7, 0)

In [8]:
model.calculateValue(100)

116.9516324728088

In [10]:
model_u = BinomialTreeModel(val_date, mat_date, 0.03, 2, 0.0125, 0.3, 0.01, 20.2, 3.7, 0)
model_d = BinomialTreeModel(val_date, mat_date, 0.03, 2, 0.0125, 0.3, 0.01, 19.8, 3.7, 0)

In [11]:
val_0 = model.calculateValue(100)
val_up = model_u.calculateValue(100)
val_down = model_d.calculateValue(100)
(val_up - val_down)/(2*val_0*0.01) 

0.3152177765922404